# Homework 4

## 1. Hashing

In [5]:
import pandas as pd
from tqdm import tqdm
import math

In [4]:
df = pd.read_csv('hash.txt', header = None)
df

,0
0,844082e02a27ddee8d99ea1af94a2969
1,ff96d6665b5c59d3a70bb8f2ba4f10be
2,b64a85884e2b159829331c19e05dbac9
3,1c8836719e84867c26ba2cfeb372c53d
4,b66f73ffd9008d9c99159e164261df51
...,...
138999995,84f48cbf9aa048ddada7e47d70ee7d47
138999996,dbce076164042cb808a0be005a307545
138999997,b175ab844ab7ec3c803a0e86a9033729
138999998,f0a6061d411a90921922eb5716caafc9


In [6]:
len(df)

139000000

In [ ]:
df.drop_duplicates(inplace=True)

In [5]:
#cardinality of the dataset
len(df)

125000000

In [ ]:
#reloading the dataframe since .drop_duplicates(inplace=True) modified the variable df
df = pd.read_csv('hash.txt', header = None)
df

In [7]:
#initializing 16 registers to 0 in order to implement the HyperLogLog algorithm
M = pd.DataFrame()
for i in range(16):
    M = M.append([0])
M.reset_index(drop=True, inplace=True)
M

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


For each row of the dataframe we compute the hash function and, given the result 'hashed_bin' of the hash function, we update the register corresponding to the decimal value of the first 4 bits of 'hashed_bin' with the max between its current value and the position of the first '1', starting from left, of the remaining 28 bits of 'hashed_bin'

In [8]:
for index, row in tqdm(df.iterrows()):
    hashed = int(row[0][0:8],16)+int(row[0][8:16],16)+int(row[0][16:24],16)+int(row[0][24:32],16)
    hashed_bin = bin(hashed%(2**32))[2:].zfill(32)
    reg_index = int(hashed_bin[0:4], 2)
    M[0][reg_index] = max(hashed_bin[4:].find('1') + 1, M[0][reg_index])
    
    

139000000it [16:34:14, 2330.06it/s]


In [10]:
#saving the values of the 16 registers
M.to_csv('registers4blocks.csv', index = False)

In [11]:
M

,0
0,24
1,23
2,27
3,23
4,22
5,22
6,23
7,23
8,23
9,28


In [12]:
#reloading the values of the 16 registers
df2 = pd.read_csv('registers4blocks.csv')
df2

,0
0,24
1,23
2,27
3,23
4,22
5,22
6,23
7,23
8,23
9,28




As the final step of the HyperLogLog algorithm, we compute the armonic mean of 2 elevated to the value of each register, then multiply it by the square of the number of registers and then we multiply it by a constant (in this case 0.673), dependant on the number of registers, to obtain the final estimate. Lastly, we compute the relative error between the estimate and the true cardinality of the dataset, that corresponds to 125000000 as we found out before.


In [13]:
sum = 0
for index, row in df2.iterrows():
    sum = sum + 1/(2**(row[0]))
estimate = (1/sum)*(16**2)*0.673
print(estimate)

116494226.3056121


In [14]:
relative_error = (estimate-125*(10**6))/(125*(10**6))
print(relative_error)

-0.06804618955510318
